In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
data_dir = 'C:/Users/sanja/OneDrive/Desktop/poultry/nameing'
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(299, 299),  
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=False
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)


Found 532 images belonging to 2 classes.
Found 132 images belonging to 2 classes.


In [5]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))


train_features = base_model.predict(train_gen)
train_labels = train_gen.classes


val_features = base_model.predict(val_gen)
val_labels = val_gen.classes


train_features_flattened = train_features.reshape(train_features.shape[0], -1)
val_features_flattened = val_features.reshape(val_features.shape[0], -1)


model = Sequential([
    Dense(1024, activation='relu', input_shape=(train_features_flattened.shape[1],)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

5/5 [==============================] - 18s 3s/step


In [6]:
history = model.fit(
    train_features_flattened, train_labels,
    validation_data=(val_features_flattened, val_labels),
    epochs=10,  
    batch_size=32
)


model.save('path_to_save_model/my_model_inceptionv310.h5')

Epoch 1/10
17/17 [==============================] - 28s 1s/step - loss: 36.1809 - accuracy: 0.6203 - val_loss: 12.6840 - val_accuracy: 0.6970
Epoch 2/10
17/17 [==============================] - 20s 1s/step - loss: 5.3753 - accuracy: 0.8647 - val_loss: 12.9580 - val_accuracy: 0.6970
Epoch 3/10
17/17 [==============================] - 20s 1s/step - loss: 2.8057 - accuracy: 0.9173 - val_loss: 8.5430 - val_accuracy: 0.7348
Epoch 4/10
17/17 [==============================] - 20s 1s/step - loss: 1.1864 - accuracy: 0.9436 - val_loss: 7.0027 - val_accuracy: 0.7424
Epoch 5/10
17/17 [==============================] - 20s 1s/step - loss: 1.3495 - accuracy: 0.9380 - val_loss: 6.3728 - val_accuracy: 0.7652
Epoch 6/10
17/17 [==============================] - 20s 1s/step - loss: 0.8276 - accuracy: 0.9680 - val_loss: 6.5852 - val_accuracy: 0.7500
Epoch 7/10
17/17 [==============================] - 19s 1s/step - loss: 0.7619 - accuracy: 0.9568 - val_loss: 4.3804 - val_accuracy: 0.7955
Epoch 8/10
17/17 

In [2]:
import numpy as np
import tkinter as tk
from tkinter import filedialog, Button, Label
from PIL import Image as PILImage, ImageTk
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from threading import Thread

model = load_model(r"C:/Users/sanja/OneDrive/Desktop/poultry/path_to_save_model/my_model_inceptionv310.h5")

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

def detect_disease(image_path, result_label, image_label):
    img = PILImage.open(image_path)
    img = img.resize((299, 299))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    features = base_model.predict(img_array)
    features_flattened = features.reshape(features.shape[0], -1)
    prediction = model.predict(features_flattened)
    predicted_class = np.round(prediction).astype(int)[0][0]
    class_names = ['Healthy', 'UnHealthy']
    result = class_names[predicted_class]

    img = PILImage.open(image_path)
    img = img.resize((250, 250))
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk
    result_label.config(text=f"Detection Result: {result}")

def evaluate_model():
   
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_gen = test_datagen.flow_from_directory(
        r'C:\Users\sanja\OneDrive\Desktop\poultry\testing', 
        target_size=(299, 299),
        batch_size=1,  
        class_mode='binary',
        shuffle=False
    )

    test_features = base_model.predict(test_gen)
    test_features_flattened = test_features.reshape(test_features.shape[0], -1)
    predictions = model.predict(test_features_flattened)
    predicted_classes = np.round(predictions).astype(int)

    true_labels = test_gen.classes
    accuracy = accuracy_score(true_labels, predicted_classes)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy

def upload_image():
   
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png *.webp ")])
    if file_path:
        detect_disease(file_path, result_label, image_label)

def on_evaluate_button_click():
    Thread(target=lambda: evaluate_model()).start()

root = tk.Tk()
root.title("Chicken Disease Detection")

label = Label(root, text="Upload an image of the chicken for disease detection:")
label.pack(pady=20)

upload_button = Button(root, text="Upload Image", command=upload_image)
upload_button.pack(pady=20)

image_label = Label(root)
image_label.pack(pady=20)

result_label = Label(root, text="Detection Result: ")
result_label.pack(pady=20)

evaluate_button = Button(root, text="Evaluate Model", command=on_evaluate_button_click)
evaluate_button.pack(pady=20)

root.mainloop()


1/1 [==============================] - 2s 2s/step
Found 17 images belonging to 2 classes.
1/1 [==============================] - 0s 320ms/step
Test Accuracy: 82.35%
